### **Section 1: Introduction to Long Short-Term Memory (LSTM)**
Long Short-Term Memory (LSTM) is a type of recurrent neural network (RNN) architecture designed to capture and remember long-term dependencies in sequential data. In this notebook, we will implement a basic LSTM model using TensorFlow and explore its components.

In [1]:
import tensorflow as tf
import numpy as np

### **Section 2: LSTM Cell Implementation**

The LSTM cell consists of several components: input gate, forget gate, cell update, and output gate. These gates control the flow of information through the cell, allowing it to capture long-term dependencies. The equations for each gate involve trainable weights and biases.

In [2]:
def lstm_cell(input_data, prev_hidden_state, prev_cell_state, weights, biases):
    """
    LSTM cell implementation for a single time step.

    Args:
    - input_data: Input data for the current time step.
    - prev_hidden_state: Hidden state from the previous time step.
    - prev_cell_state: Cell state from the previous time step.
    - weights: Dictionary of weights for the LSTM cell.
    - biases: Dictionary of biases for the LSTM cell.

    Returns:
    - hidden_state: Hidden state for the current time step.
    - cell_state: Cell state for the current time step.
    """
    # Concatenate input with previous hidden state
    concat_input = tf.concat([input_data, prev_hidden_state], axis=-1)

    # Input gate
    input_gate = tf.sigmoid(tf.matmul(concat_input, weights['input_gate']) + biases['input_gate'])

    # Forget gate
    forget_gate = tf.sigmoid(tf.matmul(concat_input, weights['forget_gate']) + biases['forget_gate'])

    # Update cell state
    cell_update = tf.tanh(tf.matmul(concat_input, weights['cell_update']) + biases['cell_update'])
    cell_state = forget_gate * prev_cell_state + input_gate * cell_update

    # Output gate
    output_gate = tf.sigmoid(tf.matmul(concat_input, weights['output_gate']) + biases['output_gate'])

    # Hidden state for the current time step
    hidden_state = output_gate * tf.tanh(cell_state)

    return hidden_state, cell_state

### **Section 3: Forward Pass through LSTM Network**

The forward pass involves processing input sequences through the LSTM cells, updating hidden states and cell states at each time step. This process allows the network to learn and capture temporal patterns in the data.

In [3]:
def lstm_forward(inputs, initial_hidden_state, initial_cell_state, weights, biases):
    """
    Forward pass through the LSTM network.

    Args:
    - inputs: List of input sequences for each time step.
    - initial_hidden_state: Initial hidden state.
    - initial_cell_state: Initial cell state.
    - weights: Dictionary of weights for the LSTM cell.
    - biases: Dictionary of biases for the LSTM cell.

    Returns:
    - hidden_states: List of hidden states for each time step.
    - cell_states: List of cell states for each time step.
    """
    hidden_states = []
    cell_states = []

    current_hidden_state = initial_hidden_state
    current_cell_state = initial_cell_state

    for input_data in inputs:
        current_hidden_state, current_cell_state = lstm_cell(input_data, current_hidden_state,
                                                             current_cell_state, weights, biases)
        hidden_states.append(current_hidden_state)
        cell_states.append(current_cell_state)

    return hidden_states, cell_states

### **Section 4: Backward Pass and Weight Updates**

The backward pass calculates the loss and updates the weights and biases using gradient descent. We use the Adam optimizer to efficiently perform these updates.

In [4]:
def lstm_backward(inputs, targets, initial_hidden_state, initial_cell_state, weights, biases, learning_rate):
    """
    Backward pass and weight updates through the LSTM network.

    Args:
    - inputs: List of input sequences for each time step.
    - targets: List of target values for each time step.
    - initial_hidden_state: Initial hidden state.
    - initial_cell_state: Initial cell state.
    - weights: Dictionary of weights for the LSTM cell.
    - biases: Dictionary of biases for the LSTM cell.
    - learning_rate: Learning rate for weight updates.
    """
    optimizer = tf.optimizers.Adam(learning_rate)

    with tf.GradientTape() as tape:
        # Forward pass to get hidden states and cell states
        hidden_states, cell_states = lstm_forward(inputs, initial_hidden_state,
                                                  initial_cell_state, weights, biases)

        # Calculate loss using the final hidden state and target values
        loss = tf.reduce_mean(tf.square(targets - hidden_states[-1]))

    # Compute gradients with respect to weights and biases
    gradients = tape.gradient(loss, list(weights.values()) + list(biases.values()))

    # Update weights and biases using the optimizer
    optimizer.apply_gradients(zip(gradients, list(weights.values()) + list(biases.values())))

### **Section 5: Training the LSTM Model**

Training the LSTM model involves iteratively performing forward and backward passes. The model learns to predict target values by adjusting its internal parameters.

In [5]:
# Function to train the LSTM model
def train_lstm(inputs, targets, initial_hidden_state, initial_cell_state, weights, biases, learning_rate, epochs):
    for epoch in range(epochs):
        lstm_backward(inputs, targets, initial_hidden_state, initial_cell_state, weights, biases, learning_rate)

        # Print details after each epoch
        if epoch % 10 == 0:
            hidden_states, _ = lstm_forward(inputs, initial_hidden_state, initial_cell_state, weights, biases)
            current_loss = tf.reduce_mean(tf.square(targets - hidden_states[-1]))

            print(f"Epoch {epoch}/{epochs}, Loss: {current_loss.numpy()}")

### **Section 6: Example Usage**

An example demonstrating the usage of the implemented LSTM model with randomly generated input sequences and targets.

In [6]:
# Example usage:
input_size = 3
hidden_size = 4
output_size = 1
sequence_length = 5
batch_size = 1  # Adjust this based on your actual batch size

# Updated weights with correct dimensions
weights = {
    'input_gate': tf.Variable(tf.random.normal([input_size + hidden_size, hidden_size])),
    'forget_gate': tf.Variable(tf.random.normal([input_size + hidden_size, hidden_size])),
    'cell_update': tf.Variable(tf.random.normal([input_size + hidden_size, hidden_size])),
    'output_gate': tf.Variable(tf.random.normal([input_size + hidden_size, hidden_size]))
}

biases = {
    'input_gate': tf.Variable(tf.zeros([hidden_size])),
    'forget_gate': tf.Variable(tf.zeros([hidden_size])),
    'cell_update': tf.Variable(tf.zeros([hidden_size])),
    'output_gate': tf.Variable(tf.zeros([hidden_size]))
}

# Generate random input sequences and targets
inputs = [tf.constant(np.random.randn(batch_size, input_size), dtype=tf.float32) for _ in range(sequence_length)]
targets = [tf.constant(np.random.randn(batch_size, output_size), dtype=tf.float32) for _ in range(sequence_length)]

initial_hidden_state = tf.Variable(tf.zeros([batch_size, hidden_size]))
initial_cell_state = tf.Variable(tf.zeros([batch_size, hidden_size]))

# Train the LSTM model
train_lstm(inputs, targets, initial_hidden_state, initial_cell_state, weights, biases, learning_rate=0.001, epochs=300)

Epoch 0/300, Loss: 1.1762666702270508
Epoch 10/300, Loss: 1.1642940044403076
Epoch 20/300, Loss: 1.1506974697113037
Epoch 30/300, Loss: 1.1352951526641846
Epoch 40/300, Loss: 1.1179077625274658
Epoch 50/300, Loss: 1.09853994846344
Epoch 60/300, Loss: 1.0772967338562012
Epoch 70/300, Loss: 1.0546246767044067
Epoch 80/300, Loss: 1.0313886404037476
Epoch 90/300, Loss: 1.0087636709213257
Epoch 100/300, Loss: 0.9880312085151672
Epoch 110/300, Loss: 0.9702739715576172
Epoch 120/300, Loss: 0.9561376571655273
Epoch 130/300, Loss: 0.9457154273986816
Epoch 140/300, Loss: 0.938938319683075
Epoch 150/300, Loss: 0.935623288154602
Epoch 160/300, Loss: 0.9351335763931274
Epoch 170/300, Loss: 0.9350149035453796
Epoch 180/300, Loss: 0.9349077939987183
Epoch 190/300, Loss: 0.934819221496582
Epoch 200/300, Loss: 0.9347406625747681
Epoch 210/300, Loss: 0.9346707463264465
Epoch 220/300, Loss: 0.9346044659614563
Epoch 230/300, Loss: 0.9345372319221497
Epoch 240/300, Loss: 0.9344624280929565
Epoch 250/300, L